In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pdf-test/AMovieRecommenderSystemMOVREC.pdf


In [2]:
   !pip install numpy transformers datasets

In [3]:
   from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

   model_name = "facebook/bart-large-cnn"  
   tokenizer = AutoTokenizer.from_pretrained(model_name)
   model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [4]:
   from datasets import load_dataset

   dataset = load_dataset("cnn_dailymail", "3.0.0", split='train')

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [5]:
   def preprocess_function(examples):
       inputs = [doc for doc in examples['article']]
       model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

       
       with tokenizer.as_target_tokenizer():
           labels = tokenizer(examples['highlights'], max_length=128, truncation=True)

       model_inputs["labels"] = labels["input_ids"]
       return model_inputs

   tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
   from transformers import TrainingArguments, Trainer

   training_args = TrainingArguments(
       output_dir="./results",
       evaluation_strategy="epoch",
       learning_rate=2e-4,
       per_device_train_batch_size=8,
       per_device_eval_batch_size=8,
       num_train_epochs=1,
       weight_decay=0.01,
       save_total_limit=3,
   )

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
   from transformers import DataCollatorForSeq2Seq

   data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

   trainer = Trainer(
       model=model,
       args=training_args,
       train_dataset=tokenized_datasets,
       eval_dataset=tokenized_datasets,
       tokenizer=tokenizer,
       data_collator=data_collator,
   )

/tmp/ipykernel_23/710649148.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
   trainer.train()

Epoch,Training Loss,Validation Loss


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


KeyboardInterrupt: 

In [15]:
model.save_pretrained("/kaggle/working/trained_model")
tokenizer.save_pretrained("/kaggle/working/trained_model")

('/kaggle/working/trained_model/tokenizer_config.json',
 '/kaggle/working/trained_model/special_tokens_map.json',
 '/kaggle/working/trained_model/vocab.json',
 '/kaggle/working/trained_model/merges.txt',
 '/kaggle/working/trained_model/added_tokens.json',
 '/kaggle/working/trained_model/tokenizer.json')

In [16]:
import torch

def summarize(text):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    model.to(device)
    
    # Tokenize the input text
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    
    
    input_ids = inputs["input_ids"].to(device)
    
 
    summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    
    
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)



In [17]:
   test_text = "Recently, the topic of Artificial Intelligence Generated Content (AIGC) has witnessed unprecedented advancements 1NExT++ Research Center, National University of Singapore, Singapore. Correspondence to: Hao Fei <haofei37@nus.edu.sg>. Proceedings of the 41 st International Conference on Machine Learning, Vienna, Austria. PMLR 235, 2024. Copyright 2024 by the author(s). with certain technologies, such as ChatGPT for text generation (OpenAI, 2022a) and diffusion models for visual generation (Fan et al., 2022). Among these, the rise of Large Language Models (LLMs) has been particularly remarkable, e.g., Flan-T5 (Chung et al., 2022), Vicuna (Chiang et al., 2023), LLaMA (Touvron et al., 2023) and Alpaca (Taori et al., 2023), showcasing their formidable human-level language reasoning and decision-making capabilities, shining a light on the path of Artificial General Intelligence (AGI). Our world is inherently multimodal, and humans perceive the world with different sensory organs for varied modal information, such as language, images, videos, and sounds, which often complement and synergize with each other. With such intuition, the purely text-based LLMs have recently been endowed with other modal understanding and perception capabilities of image, video, audio, etc. A notable approach involves employing adapters that align pre-trained encoders in other modalities to textual LLMs.This endeavor has led to the rapid development of multimodal LLMs (MM-LLMs), such as BLIP-2 (Li et al., 2023c), Flamingo (Alayrac et al., 2022), MiniGPT-4 (Zhu et al., 2023), Video-LLaMA (Zhang et al., 2023c), LLaVA (Liu et al., 2023b), PandaGPT (Su et al., 2023), and SpeechGPT (Zhang et al., 2023b). Nevertheless, most of these efforts pay attention to the multimodal content understanding at the input side. Lately, fewer works have considered multimodal generation, such as Emu (Sun et al., 2023), DREAMLLM (Dong et al., 2023), GILL (Koh et al., 2023), SEED (Ge et al., 2023). Notably, these models are confined to generating interleaved texts and images. We emphasize that natural human cognition and communication indispensably require seamless transitions between any modalities of information. This makes the exploration of any-to-any MM-LLMs critical, i.e., the ability to accept inputs in any modality and deliver responses in any appropriate modality."
   print(summarize(test_text))

NEW: "It's a very bad day for us," the victim's family says .
NEW: The victim was shot in the head, police say .
Police say the suspect, who is in critical condition in hospital, was shot .
The suspect is identified as a 17-year-old boy, who was fatally shot .


In [18]:
!pip install PyPDF2 transformers torch

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.8 MB/s eta 0:00:00


!pip install PyPDF2 transformers torch

In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch


model_path = "/kaggle/working/trained_model"  # Path where your model is saved
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/opt/conda/lib/python3.10/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [20]:
import PyPDF2

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text.split('\n\n')  

In [22]:
def summarize_paragraphs(paragraphs):
    summaries = []
    for paragraph in paragraphs:
        if paragraph.strip(): 
            inputs = tokenizer(paragraph, return_tensors="pt", max_length=1024, truncation=True)
            input_ids = inputs["input_ids"].to(device)
            summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            summaries.append(summary)
    return summaries

In [23]:
def summarize_pdf(file_path):
    paragraphs = extract_text_from_pdf(file_path)
    summaries = summarize_paragraphs(paragraphs)
    for i, summary in enumerate(summaries):
        print(f"Paragraph {i+1} Summary: {summary}\n")


pdf_file_path = "/kaggle/input/pdf-test/AMovieRecommenderSystemMOVREC.pdf"  # Replace with your PDF file path
summarize_pdf(pdf_file_path)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Paragraph 1 Summary: 



In [24]:
import PyPDF2

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text.split('\n\n')  

In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def summarize_paragraphs(paragraphs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    summaries = []
    for paragraph in paragraphs:
        if paragraph.strip(): 
            inputs = tokenizer("summarize: " + paragraph, return_tensors="pt", max_length=1024, truncation=True)
            input_ids = inputs["input_ids"].to(device)
            summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            summaries.append(summary)
    return summaries

In [26]:
def summarize_pdf(file_path):
    paragraphs = extract_text_from_pdf(file_path)
    summaries = summarize_paragraphs(paragraphs)
    for i, summary in enumerate(summaries):
        print(f"Paragraph {i+1} Summary: {summary}\n")


pdf_file_path = "/kaggle/input/pdf-test/AMovieRecommenderSystemMOVREC.pdf"
summarize_pdf(pdf_file_path)

Paragraph 1 Summary: A Movie Recommender System: MOVREC is published in the International Journal of Comput er Applic ations. It is based on collaborative filtering that makes use of the information provided by users. It then recommends the movies that is best suited to the user at that time. The recommended movie list is sorted according to the ratings given to these movies by previous users.

